# 라이브러리 가져오기

In [1]:
pip install pandas requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import requests
import gzip
from io import BytesIO

C:\Users\cmhcm\AppData\Local\Temp\ipykernel_27448\777464514.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# 덤프데이터 수집

In [3]:
# 1. 데이터 다운로드
def download_pageviews(year, month, day, hour):
    url = (
        f"https://dumps.wikimedia.org/other/pageviews/"
        f"{year}/{year}-{month:0>2}/pageviews-{year}{month:0>2}{day:0>2}-{hour:0>2}0000.gz"
    )
    response = requests.get(url)
    return response.content

# 2. 데이터 추출
def extract_gz(data):
    with gzip.open(BytesIO(data), 'rb') as f:
        return f.read().decode('utf-8')

# 3. 데이터 파싱 및 DataFrame 변환
def parse_to_dataframe(data):
    lines = data.split('\n')
    records = [line.split() for line in lines if line]
    # 데이터의 실제 컬럼 수에 맞춰 컬럼명 지정
    df = pd.DataFrame(records, columns=['domain_code', 'page_title', 'view_counts', 'another_column'])
    return df

# 4. 전체 파이프라인 실행
def fetch_pageviews_to_dataframe(year, month, day, hour):
    gz_data = download_pageviews(year, month, day, hour)
    extracted_data = extract_gz(gz_data)
    df = parse_to_dataframe(extracted_data)
    return df

# 예시 사용
year, month, day, hour = 2024, 7, 30, 0
df = fetch_pageviews_to_dataframe(year, month, day, hour)

## 데이터 정제

In [4]:
df.columns = ['LanguageCode', 'PageTitle', 'PageViews', 'Metadata']

In [5]:
# Metadata 컬럼 DROP
df.drop(columns=['Metadata'], inplace=True)

In [6]:
# 결측치 또는 비어있는 값이 있다면 처리 (예: NaN을 0으로 대체)
df['PageViews'] = df['PageViews'].fillna(0)
# object 타입을 int로 변환
df['PageViews'] = df['PageViews'].astype(int)

In [7]:
# 임계값 100
# 100 이상의 페이지뷰만 남기기 -> 너무 적은 조회수는 담지 않기로함 (용량 문제)
threshold = 100
df_filtered = df[df['PageViews'] >= threshold]

In [8]:
# 페이지 유형에 따른 필터링 기준 설정
# 아직 더 구체화해야 함
exclude_prefixes = [
    "Main_Page", "Special:", "Talk:", "User:", "User talk:", 
    "Category:", "Portal:", "File:", "Template:", "Help:", "Wikipedia:",
    "MediaWiki:", "Spécial:", "メインページ"
]

# 특정 유형의 페이지 제외
# PageTitle이 제외할 접두사를 가진 경우 제외
df_filtered = df_filtered[~df_filtered['PageTitle'].str.startswith(tuple(exclude_prefixes))]

In [9]:
# 언어 코드별 페이지 수 확인
language_counts = df_filtered['LanguageCode'].value_counts()
# 문서 수가 10 이하인 언어 코드 찾기
languages_to_remove = language_counts[language_counts <= 10].index
# 해당 언어 코드를 가진 행 제거
df_filtered = df_filtered[~df_filtered['LanguageCode'].isin(languages_to_remove)]

In [10]:
# 나라 딕셔너리 생성
language_to_country = {
    'en': 'United States',
    'ar': 'Saudi Arabia',
    'zh': 'China',
    'es': 'Spain',
    'fr': 'France',
    'de': 'Germany',
    'ru': 'Russia',
    'ko': 'Korea',
    'ja': 'Japan',
    'fa': 'Iran',
    'vi': 'Vietnam',
    'he': 'Israel',        
    'id': 'Indonesia',  
    'it': 'Italy',       
    'pl': 'Poland',      
    'pt': 'Portugal',     
    'tl': 'Philippines', 
    'tr': 'Turkey',     
    # 다른 언어 코드와 나라를 추가할 수 있습니다
}

In [11]:
# LanguageCode에서 언어 코드를 추출하여 Country 컬럼 생성
df_filtered['Country'] = df_filtered['LanguageCode'].str.split('.').str[0].map(language_to_country)

In [12]:
df_filtered

,LanguageCode,PageTitle,PageViews,Country
14545,ar,الصفحة_الرئيسة,162,Saudi Arabia
21671,ar,تصنيف:جميع_المقالات_المقترح_دمجها,341,Saudi Arabia
24207,ar,تصنيف:مقالات_يتيمة_منذ_أبريل_2017,342,Saudi Arabia
24208,ar,تصنيف:مقالات_يتيمة_منذ_أبريل_2018,335,Saudi Arabia
24209,ar,تصنيف:مقالات_يتيمة_منذ_أبريل_2019,338,Saudi Arabia
...,...,...,...,...
5288966,zh.m,零日攻擊_(電視劇),115,China
5290192,zh.m,颱風凱米_(2024年),107,China
5292023,zh.m,高虹安,120,China
5293401,zh.m,黃道十二宮,113,China


In [13]:
# URL 컬럼 생성
def create_url(row):
    page_title = row['PageTitle'].replace('_', ' ')
    return f"https://{row['LanguageCode']}.wikipedia.org/wiki/{page_title}"

df_filtered['URL'] = df_filtered.apply(create_url, axis=1)


In [14]:
df_filtered

,LanguageCode,PageTitle,PageViews,Country,URL
14545,ar,الصفحة_الرئيسة,162,Saudi Arabia,https://ar.wikipedia.org/wiki/الصفحة الرئيسة
21671,ar,تصنيف:جميع_المقالات_المقترح_دمجها,341,Saudi Arabia,https://ar.wikipedia.org/wiki/تصنيف:جميع المقا...
24207,ar,تصنيف:مقالات_يتيمة_منذ_أبريل_2017,342,Saudi Arabia,https://ar.wikipedia.org/wiki/تصنيف:مقالات يتي...
24208,ar,تصنيف:مقالات_يتيمة_منذ_أبريل_2018,335,Saudi Arabia,https://ar.wikipedia.org/wiki/تصنيف:مقالات يتي...
24209,ar,تصنيف:مقالات_يتيمة_منذ_أبريل_2019,338,Saudi Arabia,https://ar.wikipedia.org/wiki/تصنيف:مقالات يتي...
...,...,...,...,...,...
5288966,zh.m,零日攻擊_(電視劇),115,China,https://zh.m.wikipedia.org/wiki/零日攻擊 (電視劇)
5290192,zh.m,颱風凱米_(2024年),107,China,https://zh.m.wikipedia.org/wiki/颱風凱米 (2024年)
5292023,zh.m,高虹安,120,China,https://zh.m.wikipedia.org/wiki/高虹安
5293401,zh.m,黃道十二宮,113,China,https://zh.m.wikipedia.org/wiki/黃道十二宮


In [17]:
df_filtered["URL"].head(100)

14545         https://ar.wikipedia.org/wiki/الصفحة الرئيسة
21671    https://ar.wikipedia.org/wiki/تصنيف:جميع المقا...
24207    https://ar.wikipedia.org/wiki/تصنيف:مقالات يتي...
24208    https://ar.wikipedia.org/wiki/تصنيف:مقالات يتي...
24209    https://ar.wikipedia.org/wiki/تصنيف:مقالات يتي...
                               ...                        
92028    https://ar.m.wikipedia.org/wiki/عبد العزيز بن ...
94094      https://ar.m.wikipedia.org/wiki/علي بن أبي طالب
94406           https://ar.m.wikipedia.org/wiki/عمر الراضي
94667    https://ar.m.wikipedia.org/wiki/عملية طوفان ال...
94897    https://ar.m.wikipedia.org/wiki/عيد العرش (الم...
Name: URL, Length: 100, dtype: object

In [ ]:
## 제목을 두고 양옆 공백을 삭제하는 함수 하나 strip
## 제목을 공백을 언더스코어에서 공백으로 대체하는 함수 하나 replace
## if문을 통해 ar 일시 공백을 언더스코어로  replace
## 아 애초에 ar이 아니면 공백으로 replace로 하자